**Enable autoreload extension**

In [1]:
%load_ext autoreload
%autoreload 3

**Setup dependencies**

In [2]:
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
%pip install git+https://github.com/huawei-noah/SMARTS.git@2845a86a6fa88072a196095fe04b2d0ad90b06c4#egg=smarts[camera-obs]
!rm -rf /content/SMARTS
!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

**Import Base Modules**

In [3]:
import gym

import smarts
import smarts.core
import smarts.env

**Define a new agent for this example**

In [4]:
from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

**Create an Environment (likely elsewhere)**

In [5]:
from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.env.wrappers.single_agent import SingleAgent

agent_spec = AgentSpec(
    interface=AgentInterface.from_type(
        AgentType.LanerWithSpeed, max_episode_steps=100,
    ),
    agent_builder=ChaseViaPointsAgent
)

def entry_point(*args, **kwargs):
  from smarts.env.hiway_env import HiWayEnv
  scenario = "/content/SMARTS/scenarios/figure_eight"
  ## Note: can build the scenario here
  from cli.studio import _build_single_scenario
  _build_single_scenario(clean=True, allow_offset_map=True, scenario=scenario)
  return SingleAgent(
    HiWayEnv(
      agent_specs={"agent-007": agent_spec},
      scenarios=[scenario],
      headless=True,
      sumo_headless=True,
  ))

gym.register(
    "figure_eight-v1",
    entry_point=entry_point
)

**Run an episode**

In [6]:
from smarts.core.utils.episodes import episodes


env = gym.make("figure_eight-v1")

for episode in episodes(n=1):
    agent = agent_spec.build_agent()
    observation = env.reset()
    episode.record_scenario(env.scenario_log)

    done = False
    while not done:
        agent_action = agent.act(observation)
        observation, reward, done, info = env.step(agent_action)
        episode.record_step(observation, reward, done, info)

env.close()
print ('Success')

/usr/local/lib/python3.7/dist-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


build-scenario /content/SMARTS/scenarios/figure_eight
╭────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────╮
│            Episode │     Sim T / Wall T │        Total Steps │        Steps / Sec │       Scenario Map │    Scenario Routes │     Mission (Hash) │             Scores │
├────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤


/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

│                0/1 │               3.42 │                 99 │              34.22 │       figure_eight │                    │ -19697664969423860 │ 133.93 - SingleAgent │
╰────────────────────┴────────────────────┴────────────────────┴────────────────────┴────────────────────┴────────────────────┴────────────────────┴────────────────────╯
Success
